In [ ]:
import torch
from torch import nn
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plot

# 1. ***loss function bce***
***F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)***
- weight (Tensor, optional): a manual rescaling weight given to the loss of each batch element.
- If given, has to be a Tensor of size `nbatch`.

***The unreduced (i.e. with `reduction` set to ``'none'``) loss can be described as***

$$
\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
l_n = - w_n \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right],
$$

`N` is the batch size.

***If `reduction` is not ``'none'`` (default ``'mean'``), then***

$$
\ell(x, y) = \begin{cases}
\operatorname{mean}(L), \quad & \text{if reduction} = \text{`mean';}\\
\operatorname{sum}(L), \quad & \text{if reduction} = \text{`sum'.}
\end{cases}
$$

In [ ]:
bce = nn.BCELoss()

def binary_cross_entorpy_np(inputs, targets):
    inputs = inputs.numpy()
    inputs = inputs.reshape((inputs.shape[0]*inputs.shape[1], inputs.shape[-2]*inputs.shape[-1]))
    targets = targets.numpy()
    targets = targets.reshape((targets.shape[0]*targets.shape[1], targets.shape[-2]*targets.shape[-1]))
    outputs = 0.
    weight = 1.
    for i in range(targets.shape[0]):
        temp = 0
        for j in range(targets.shape[1]):
            temp += -1. * weight * (targets[i, j]*np.log(inputs[i, j]) + (1-targets[i, j])*np.log(1-inputs[i, j]))
        outputs += (temp / targets.shape[1])
    return np.around(outputs / targets.shape[0], 4)

In [ ]:
a = torch.rand((1, 2, 2, 2))
b = torch.tensor([[[[0, 1.], [1., 0]], [[0, 1.], [1., 0]]]])
binary_cross_entorpy_np(a, b)
print(a, b, bce(a, b), binary_cross_entorpy_np(a, b), sep="\n")

# 2. ***loss function ce***
***F.cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index, reduction=self.reduction, label_smoothing=self.label_smoothing)***
- this case is equivalent to the combination of `~torch.nn.LogSoftmax` and `~torch.nn.NLLLoss`.
- It is useful when training a classification problem with `C` classes.
- If provided, the optional argument `weight` should be a 1D `Tensor` assigning weight to each of the classes, This is
particularly useful when you have an unbalanced training set.

***The `input` is expected to contain raw, unnormalized scores for each class.***
- `input` has to be a Tensor of size `(C)` for unbatched input, `(minibatch, C)` or `(minibatch, C, d_1, d_2, ..., d_K)` with `K
\geq 1` for the `K`-dimensional case.
- The last being useful for higher dimension inputs, such as computing cross entropy loss per-pixel for 2D images.

***The `target` that this criterion expects should contain either***
- Class indices in the range `[0, C)` where `C` is the number of classes, not one-hot.
- if `ignore_index` is specified, loss also accepts this class index (this index may not necessarily be in the class range).
- If containing class probabilities, same shape as the input and each value should be between `[0, 1]`.
- The unreduced (i.e. with `reduction` set to ``'none'``) loss for this case can be described as

$$
\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
l_n = - w_{y_n} \log \frac{\exp(x_{n,y_n})}{\sum_{c=1}^C \exp(x_{n,c})}
\cdot \mathbb{1}\{y_n \not= \text{ignore\_index}\}
$$

`x` is the input, `y` is the target, `w` is the weight,
`C` is the number of classes, and `N` spans the minibatch dimension as well as `d_1, ..., d_k` for the `K`-dimensional case.

***If `reduction` is not ``'none'`` (default ``'mean'``), then***

$$
\ell(x, y) = \begin{cases}
  \sum_{n=1}^N \frac{1}{\sum_{n=1}^N w_{y_n} \cdot \mathbb{1}\{y_n \not= \text{ignore\_index}\}} l_n, \quad &
   \text{if reduction} = \text{`mean';}\\
    \sum_{n=1}^N l_n, \quad &
    \text{if reduction} = \text{`sum'.}
\end{cases}
$$

- Probabilities for each class; useful when labels beyond a single class per minibatch item
  are required, such as for blended labels, label smoothing, etc. The unreduced (i.e. with
  `reduction` set to ``'none'``) loss for this case can be described as

$$
\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
l_n = - \sum_{c=1}^C w_c \log \frac{\exp(x_{n,c})}{\sum_{i=1}^C \exp(x_{n,i})} y_{n,c}
$$

`x` is the input, `y` is the target, `w` is the weight,
`C` is the number of classes, and `N` spans the minibatch dimension as well as `d_1, ..., d_k` for the `K`-dimensional case.

***If `reduction` is not ``'none'`` (default ``'mean'``), then***

$$
\ell(x, y) = \begin{cases}
  \frac{\sum_{n=1}^N l_n}{N}, \quad &
   \text{if reduction} = \text{`mean';}\\
    \sum_{n=1}^N l_n, \quad &
    \text{if reduction} = \text{`sum'.}
\end{cases}
$$

- The performance of this criterion is generally better when `target` contains class
  indices, as this allows for optimized computation. Consider providing `target` as
  class probabilities only when a single class label per minibatch item is too restrictive.

***The `output` If reduction is 'none', same shape as the target. Otherwise, scalar.***

In [ ]:
ce = nn.CrossEntropyLoss()

ls = nn.LogSoftmax(dim=1)
nll = nn.NLLLoss()

def cross_entorpy_np(inputs, targets):
    inputs = inputs.numpy()
    inputs = inputs.reshape((inputs.shape[0], inputs.shape[1], inputs.shape[-2]*inputs.shape[-1]))
    targets = targets.numpy()
    targets = targets.reshape((targets.shape[0], targets.shape[1], targets.shape[-2]*targets.shape[-1]))
    outputs = 0.
    weight = 1.
    for i in range(targets.shape[0]):
        temp = 0
        for j in range(targets.shape[1]):
            temp += -1. * weight * np.log((np.exp(inputs[i, j, :])[targets[i, j, :]] / np.sum(np.exp(inputs[i, j, :]))))
        outputs += (temp / targets.shape[1])
    return np.around(outputs / targets.shape[0], 4)

In [ ]:
inputs = torch.Tensor([[[[0.1234, 0.5555, 0.3211]], [[0.1234, 0.5555, 0.3211]], [[0.1234, 0.5555, 0.3211]]]])
targets = torch.tensor([[[[0, 1, 2]]]])

outputs = ce(inputs, targets)
print(outputs)

# outputs = nll(ls(inputs), targets)
# print(outputs)

outputs = cross_entorpy_np(inputs, targets)
print(outputs)

In [ ]:
a = np.random.rand(1, 2, 2, 2)

In [ ]:
a.flatten((-2, -1))

In [ ]:
b = a.reshape(2, 4)
b.shape
b.shape[0]

In [ ]:
np.log(0.5)

In [ ]:
input=torch.Tensor([[0.1234, 0.5555,0.3211],[0.1234, 0.5555,0.3211],[0.1234, 0.5555,0.3211],])
target = torch.tensor([0,1,2])
input[0][target[0]]

In [ ]:
generator = torch.Generator(device="cpu").manual_seed(51)

In [ ]:
torch.rand(5, generator=generator)

In [ ]:
torch.rand(5, generator=generator)

In [18]:
generator = torch.Generator(device="cpu").manual_seed(51)

In [26]:
torch.rand(5, generator=generator)

tensor([0.9486, 0.0251, 0.1548, 0.4304, 0.5786])

In [23]:
torch.rand(5, generator=generator)

tensor([0.6319, 0.9800, 0.8547, 0.7153, 0.5669])